In [ ]:
```json
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# ContextualConvoCondenser: Data Exploration"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Setup and Data Loading"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import nltk\n",
    "from nltk.sentiment.vader import SentimentIntensityAnalyzer\n",
    "from nltk.tokenize import word_tokenize, sent_tokenize\n",
    "from nltk.probability import FreqDist\n",
    "from wordcloud import WordCloud\n",
    "import spacy\n",
    "from collections import Counter\n",
    "import re\n",
    "import warnings\n",
    "\n",
    "# Suppress specific warnings for cleaner output\n",
    "warnings.filterwarnings('ignore', category=FutureWarning)\n",
    "warnings.filterwarnings('ignore', category=UserWarning)\n",
    "\n",
    "# Download necessary NLTK data (if not already downloaded)\n",
    "try:\n",
    "    nltk.data.find('sentiment/vader_lexicon.zip')\n",
    "except nltk.downloader.DownloadError:\n",
    "    nltk.download('vader_lexicon')\n",
    "try:\n",
    "    nltk.data.find('tokenizers/punkt')\n",
    "except nltk.downloader.DownloadError:\n",
    "    nltk.download('punkt')\n",
    "\n",
    "# Load spaCy model (download if needed)\n",
    "try:\n",
    "    nlp = spacy.load('en_core_web_sm')\n",
    "except OSError:\n",
    "    print('Downloading spaCy en_core_web_sm model...')\n",
    "    spacy.cli.download('en_core_web_sm')\n",
    "    nlp = spacy.load('en_core_web_sm')\n",
    "\n",
    "# Set plot style\n",
    "sns.set_style('whitegrid')\n",
    "plt.rcParams['figure.figsize'] = (12, 6)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Create Synthetic Sample Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create synthetic conversation data representing transcripts\n",
    "data = {\n",
    "    'conversation_id': ['conv_001'] * 8 + ['conv_002'] * 10 + ['conv_003'] * 7,\n",
    "    'timestamp': pd.to_datetime([\n",
    "        '2023-10-26 09:01:00', '2023-10-26 09:01:30', '2023-10-26 09:02:15', '2023-10-26 09:03:00',\n",
    "        '2023-10-26 09:03:45', '2023-10-26 09:04:30', '2023-10-26 09:05:00', '2023-10-26 09:05:45',\n",
    "        '2023-10-27 14:00:00', '2023-10-27 14:00:45', '2023-10-27 14:01:30', '2023-10-27 14:02:00',\n",
    "        '2023-10-27 14:02:45', '2023-10-27 14:03:30', '2023-10-27 14:04:15', '2023-10-27 14:05:00',\n",
    "        '2023-10-27 14:05:45', '2023-10-27 14:06:30',\n",
    "        '2023-10-28 11:30:00', '2023-10-28 11:30:40', '2023-10-28 11:31:15', '2023-10-28 11:32:05',\n",
    "        '2023-10-28 11:32:50', '2023-10-28 11:33:30', '2023-10-28 11:34:00'\n",
    "    ]),\n",
    "    'speaker': [\n",
    "        'Alice', 'Bob', 'Alice', 'Charlie', 'Bob', 'Alice', 'Charlie', 'Bob', # conv_001\n",
    "        'SupportAgent', 'Customer', 'SupportAgent', 'Customer', 'SupportAgent', 'Customer', 'SupportAgent', 'Customer', 'SupportAgent', 'Customer', # conv_002\n",
    "        'David', 'Eve', 'David', 'Eve', 'Frank', 'David', 'Eve' # conv_003\n",
    "    ],\n",
    "    'utterance': [\n",
    "        \"Okay team, let's kick off the project planning.\", \"Agreed. First, we need to define the key milestones.\", \"Right. Milestone 1 should be the requirements gathering, due next Friday.\", \"I can take the lead on that. I'll draft the document.\", \"Thanks, Charlie. Bob, can you handle the resource allocation plan?\", \"Sure, I'll get that done by Wednesday.\", \"Excellent. Any initial concerns?\", \"Just the tight deadline for Milestone 1, but it's doable.\", # conv_001\n",
    "        \"Hello, thank you for calling Tech Support. How can I help you?\", \"Hi, my internet connection keeps dropping.\", \"I understand that must be frustrating. Can you tell me when this started?\", \"It's been happening intermittently for the past two days.\", \"Okay, let's try resetting your modem. Could you unplug it for 30 seconds?\", \"Alright, doing that now... Okay, it's plugged back in.\", \"Great. Please give it a minute or two to reconnect. Is the 'Internet' light solid now?\", \"Yes, it looks stable for now.\", \"Good. If the issue persists, please call us back with reference number 12345. Is there anything else?\", \"No, that's all. Thank you!\", # conv_002\n",
    "        \"Morning everyone. Let's discuss the Q3 results.\", \"Morning. Overall, sales are up 15% year-over-year, which is fantastic.\", \"Great news! Which product line drove most of that growth?\", \"The new 'Gadget Pro' accounted for nearly 60% of the increase.\", \"Impressive. Frank, any updates from marketing on the Gadget Pro campaign?\", \"Yes, the digital campaign exceeded targets. We need to decide on the budget for Q4 continuation.\", \"Okay, let's schedule a follow-up on that. Eve, please set up a meeting for next Tuesday.\" # conv_003\n",
    "    ]\n",
    "}\n",
    "\n",
    "df = pd.DataFrame(data)\n",
    "\n",
    "# Combine utterances into full transcripts for each conversation\n",
    "transcripts = df.groupby('conversation_id').agg(\n",
    "    full_transcript=('utterance', lambda x: ' '.join(x)),\n",
    "    start_time=('timestamp', 'min'),\n",
    "    end_time=('timestamp', 'max'),\n",
    "    num_utterances=('utterance', 'count'),\n",
    "    speakers=('speaker', lambda x: list(x.unique()))\n",
    ").reset_index()\n",
    "\n",
    "transcripts['duration'] = transcripts['end_time'] - transcripts['start_time']"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Data Inspection"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"Utterance Data Head:\")\n",
    "print(df.head())\n",
    "print(\"\\nUtterance Data Info:\")\n",
    "df.info()\n",
    "print(\"\\nUtterance Data Missing Values:\")\n",
    "print(df.isnull().sum())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"\\nTranscript Data Head:\")\n",
    "print(transcripts.head())\n",
    "print(\"\\nTranscript Data Info:\")\n",
    "transcripts.info()\n",
    "print(\"\\nTranscript Data Missing Values:\")\n",
    "print(transcripts.isnull().sum())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Exploratory Data Analysis (EDA)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Distribution of Utterance Length"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "df['utterance_length'] = df['utterance'].apply(lambda x: len(word_tokenize(x)))\n",
    "\n",
    "plt.figure(figsize=(10, 5))\n",
    "sns.histplot(df['utterance_length'], bins=15, kde=True)\n",
    "plt.title('Distribution of Utterance Length (Number of Words)')\n",
    "plt.xlabel('Number of Words')\n",
    "plt.ylabel('Frequency')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Distribution of Transcript Length"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "transcripts['transcript_length'] = transcripts['full_transcript'].apply(lambda x: len(word_tokenize(x)))\n",
    "\n",
    "plt.figure(figsize=(10, 5))\n",
    "sns.histplot(transcripts['transcript_length'], bins=5, kde=False)\n",
    "plt.title('Distribution of Full Transcript Length (Number of Words)')\n",
    "plt.xlabel('Number of Words')\n",
    "plt.ylabel('Frequency')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Speaker Contribution"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(12, 6))\n",
    "sns.countplot(data=df, y='speaker', order=df['speaker'].value_counts().index, palette='viridis')\n",
    "plt.title('Number of Utterances per Speaker')\n",
    "plt.xlabel('Number of Utterances')\n",
    "plt.ylabel('Speaker')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Word Cloud of All Transcripts"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "all_text = ' '.join(transcripts['full_transcript'])\n",
    "wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_text)\n",
    "\n",
    "plt.figure(figsize=(15, 7))\n",
    "plt.imshow(wordcloud, interpolation='bilinear')\n",
    "plt.axis('off')\n",
    "plt.title('Word Cloud for All Transcripts')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Conversation Duration Distribution"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Convert duration to total seconds for plotting\n",
    "transcripts['duration_seconds'] = transcripts['duration'].dt.total_seconds()\n",
    "\n",
    "plt.figure(figsize=(10, 5))\n",
    "sns.histplot(transcripts['duration_seconds'], bins=5, kde=False)\n",
    "plt.title('Distribution of Conversation Duration')\n",
    "plt.xlabel('Duration (seconds)')\n",
    "plt.ylabel('Frequency')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Statistical Analysis"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Basic Text Statistics"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Average utterance length per conversation\n",
    "avg_utterance_len = df.groupby('conversation_id')['utterance_length'].mean()\n",
    "transcripts = transcripts.merge(avg_utterance_len.rename('avg_utterance_len'), on='conversation_id')\n",
    "\n",
    "# Vocabulary richness (Type-Token Ratio) per conversation\n",
    "def calculate_ttr(text):\n",
    "    tokens = [token.lower() for token in word_tokenize(text) if token.isalpha()]\n",
    "    if not tokens:\n",
    "        return 0\n",
    "    return len(set(tokens)) / len(tokens)\n",
    "\n",
    "transcripts['ttr'] = transcripts['full_transcript'].apply(calculate_ttr)\n",
    "\n",
    "print(\"Transcript Statistics:\")\n",
    "print(transcripts[['conversation_id', 'num_utterances', 'transcript_length', 'avg_utterance_len', 'ttr', 'duration_seconds']].describe())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Most Common Words (excluding stopwords)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from nltk.corpus import stopwords\n",
    "\n",
    "# Download stopwords if necessary\n",
    "try:\n",
    "    nltk.data.find('corpora/stopwords')\n",
    "except nltk.downloader.DownloadError:\n",
    "    nltk.download('stopwords')\n",
    "\n",
    "stop_words = set(stopwords.words('english'))\n",
    "\n",
    "all_tokens = word_tokenize(all_text.lower())\n",
    "filtered_tokens = [word for word in all_tokens if word.isalpha() and word not in stop_words]\n",
    "\n",
    "fdist = FreqDist(filtered_tokens)\n",
    "\n",
    "print(\"\\nTop 20 Most Common Words (excluding stopwords):\")\n",
    "print(fdist.most_common(20))\n",
    "\n",
    "# Plot frequency distribution\n",
    "plt.figure(figsize=(12, 6))\n",
    "fdist.plot(20, cumulative=False)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Feature Engineering Experiments"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Sentiment Analysis per Utterance"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "sid = SentimentIntensityAnalyzer()\n",
    "\n",
    "df['sentiment_scores'] = df['utterance'].apply(lambda x: sid.polarity_scores(x))\n",
    "df['sentiment_compound'] = df['sentiment_scores'].apply(lambda x: x['compound'])\n",
    "df['sentiment_label'] = df['sentiment_compound'].apply(lambda c: 'positive' if c >= 0.05 else ('negative' if c <= -0.05 else 'neutral'))\n",
    "\n",
    "print(\"Sentiment Analysis per Utterance (Sample):\")\n",
    "print(df[['utterance', 'sentiment_compound', 'sentiment_label']].head())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualize sentiment distribution\n",
    "plt.figure(figsize=(8, 5))\n",
    "sns.countplot(data=df, x='sentiment_label', palette={'positive':'green', 'neutral':'grey', 'negative':'red'})\n",
    "plt.title('Distribution of Utterance Sentiment')\n",
    "plt.xlabel('Sentiment Label')\n",
    "plt.ylabel('Count')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Sentiment Trend within Conversations ('Vibe Check')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(15, 7))\n",
    "for conv_id in df['conversation_id'].unique():\n",
    "    conv_df = df[df['conversation_id'] == conv_id].reset_index()\n",
    "    plt.plot(conv_df.index, conv_df['sentiment_compound'], marker='o', linestyle='-', label=conv_id)\n",
    "\n",
    "plt.title('Sentiment Trend Over Utterances within Conversations')\n",
    "plt.xlabel('Utterance Sequence')\n",
    "plt.ylabel('Sentiment Compound Score')\n",
    "plt.legend(title='Conversation ID')\n",
    "plt.axhline(0.05, color='green', linestyle='--', alpha=0.5, label='Positive Threshold')\n",
    "plt.axhline(-0.05, color='red', linestyle='--', alpha=0.5, label='Negative Threshold')\n",
    "plt.ylim(-1.1, 1.1)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Named Entity Recognition (NER) - Example"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def extract_entities(text):\n",
    "    doc = nlp(text)\n",
    "    entities = [(ent.text, ent.label_) for ent in doc.ents]\n",
    "    return entities\n",
    "\n",
    "# Apply NER to a sample transcript\n",
    "sample_transcript = transcripts.loc[0, 'full_transcript']\n",
    "sample_entities = extract_entities(sample_transcript)\n",
    "\n",
    "print(f\"Entities for Conversation {transcripts.loc[0, 'conversation_id']}:\")\n",
    "print(sample_entities)\n",
    "\n",
    "# Count entity types across all transcripts\n",
    "all_entities = []\n",
    "for transcript in transcripts['full_transcript']:\n",
    "    all_entities.extend(extract_entities(transcript))\n",
    "\n",
    "entity_labels = [label for text, label in all_entities]\n",
    "entity_counts = Counter(entity_labels)\n",
    "\n",
    "print(\"\\nOverall Entity Type Counts:\")\n",
    "print(entity_counts)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Visualize top N entity types\n",
    "common_entities = entity_counts.most_common(10)\n",
    "labels, values = zip(*common_entities)\n",
    "\n",
    "plt.figure(figsize=(12, 6))\n",
    "sns.barplot(x=list(values), y=list(labels), palette='mako')\n",
    "plt.title('Top 10 Most Common Entity Types Across All Transcripts')\n",
    "plt.xlabel('Frequency')\n",
    "plt.ylabel('Entity Type')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Keyword Spotting (Action Items / Decisions)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Simple keyword spotting for potential action items or decisions\n",
    "action_keywords = ['i will', 'i\'ll', 'can you', 'assign', 'task', 'action item', 'due', 'deadline', 'follow-up', 'schedule']\n",
    "decision_keywords = ['agreed', 'decide', 'decision', 'plan is', 'we need to', 'confirm', 'okay', 'right']\n",
    "\n",
    "def count_keywords(text, keywords):\n",
    "    count = 0\n",
    "    text_lower = text.lower()\n",
    "    for keyword in keywords:\n",
    "        count += len(re.findall(r'\\b' + re.escape(keyword) + r'\\b', text_lower))\n",
    "    return count\n",
    "\n",
    "df['action_keyword_count'] = df['utterance'].apply(lambda x: count_keywords(x, action_keywords))\n",
    "df['decision_keyword_count'] = df['utterance'].apply(lambda x: count_keywords(x, decision_keywords))\n",
    "\n",
    "transcripts['total_action_keywords'] = df.groupby('conversation_id')['action_keyword_count'].sum()\n",
    "transcripts['total_decision_keywords'] = df.groupby('conversation_id')['decision_keyword_count'].sum()\n",
    "\n",
    "print(\"Keyword Counts per Transcript:\")\n",
    "print(transcripts[['conversation_id', 'total_action_keywords', 'total_decision_keywords']])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Initial Model Testing"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Basic Summarization with Hugging Face Transformers"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Note: This requires the `transformers` and `torch` (or `tensorflow`) libraries\n",
    "# pip install transformers torch\n",
    "from transformers import pipeline\n",
    "\n",
    "# Load a pre-trained summarization pipeline (using a small model for speed)\n",
    "# Using BART as an example, T5 is another good option.\n",
    "try:\n",
    "    summarizer = pipeline(\"summarization\", model=\"sshleifer/distilbart-cnn-6-6\", device=-1) # Use CPU\n",
    "    print(\"Summarization pipeline loaded successfully.\")\n",
    "except Exception as e:\n",
    "    print(f\"Error loading summarization pipeline: {e}\")\n",
    "    print(\"Skipping summarization test. Ensure 'transformers' and a backend (torch/tensorflow) are installed.\")\n",
    "    summarizer = None\n",
    "\n",
    "if summarizer:\n",
    "    # Select a sample transcript to summarize\n",
    "    sample_conv_id = 'conv_001'\n",
    "    text_to_summarize = transcripts[transcripts['conversation_id'] == sample_conv_id]['full_transcript'].iloc[0]\n",
    "\n",
    "    print(f\"\\nOriginal Transcript ({sample_conv_id}):\")\n",
    "    print(text_to_summarize)\n",
    "\n",
    "    # Generate summary (adjust max/min length as needed)\n",
    "    # Note: Summarization models have input length limits. Longer texts might need chunking.\n",
    "    try:\n",
    "        summary = summarizer(text_to_summarize, max_length=100, min_length=25, do_sample=False)[0]['summary_text']\n",
    "        print(\"\\nGenerated Summary:\")\n",
    "        print(summary)\n",
    "    except Exception as e:\n",
    "        print(f\"\\nError during summarization: {e}\")\n",
    "        # Common issue: Input text exceeds model's maximum sequence length.\n",
    "        if \"maximum sequence length\" in str(e):\n",
    "             print(\"Input text might be too long for this model. Consider chunking or using a model with a larger context window.\")\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Sentiment Analysis with Hugging Face Transformers (Alternative)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Using a transformer-based sentiment analysis pipeline\n",
    "try:\n",
    "    sentiment_pipeline = pipeline(\"sentiment-analysis\", model=\"distilbert-base-uncased-finetuned-sst-2-english\", device=-1) # Use CPU\n",
    "    print(\"\\nSentiment analysis pipeline loaded successfully.\")\n",
    "except Exception as e:\n",
    "    print(f\"\\nError loading sentiment analysis pipeline: {e}\")\n",
    "    print(\"Skipping transformer sentiment test. Ensure 'transformers' and a backend (torch/tensorflow) are installed.\")\n",
    "    sentiment_pipeline = None\n",
    "\n",
    "if sentiment_pipeline:\n",
    "    # Analyze sentiment of a few sample utterances\n",
    "    sample_utterances = df['utterance'].head(3).tolist()\n",
    "    print(\"\\nAnalyzing sentiment of sample utterances:\")\n",
    "    for utt in sample_utterances:\n",
    "        try:\n",
    "            result = sentiment_pipeline(utt)[0]\n",
    "            print(f\"- Utterance: '{utt}'\")\n",
    "            print(f\"  Sentiment: {result['label']}, Score: {result['score']:.4f}\")\n",
    "        except Exception as e:\n",
    "            print(f\"\\nError during sentiment analysis for utterance '{utt}': {e}\")\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## End of Exploration"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"